In [82]:
import sys
sys.path.append('/usr/lib/python2.7/dist-packages')

# chapter 3  Cumulative Distribution Functions

Objective is to understand CDFs as an alternative representation of distributions with many members, relation between percentiles (and other rank based metrics such as median, IQR) and CDFs, using CDFs for resampling, etc.

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

## Class Size Paradox

Given a probability distribution, the mean of a distribution calculated from its PMF is lower than if we were to calculate the mean by taking a sample from it. This happens because the larger classes tend to get oversampled.

In [31]:
class_size_dist = {
    7: 8,
    12: 8,
    17: 14,
    22: 4,
    27: 6,
    32: 12,
    37: 8,
    42: 3,
    47: 2
}
class_sizes = []
sum = 0
for k in class_size_dist.keys():
    num_k = class_size_dist[k]
    for i in range(num_k):
        class_sizes.append(k)
print(class_sizes)
print(len(class_sizes))
for k in class_size_dist.keys():
    print(class_size_dist[k]/65)
    sum = sum + (class_size_dist[k]/65)
print(sum)



[7, 7, 7, 7, 7, 7, 7, 7, 12, 12, 12, 12, 12, 12, 12, 12, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 22, 22, 22, 22, 27, 27, 27, 27, 27, 27, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 37, 37, 37, 37, 37, 37, 37, 37, 42, 42, 42, 47, 47]
65
0.12307692307692308
0.12307692307692308
0.2153846153846154
0.06153846153846154
0.09230769230769231
0.18461538461538463
0.12307692307692308
0.046153846153846156
0.03076923076923077
1.0


In [54]:
pmf_class_sizes = np.histogram(np.array(class_sizes),bins = 40, range=(7,47), normed = (True))
def pmf_mean(pmf):
    print(pmf[0])
    print(pmf[1])
    ps = pmf[0]
    xs = pmf[1][:-1]
    return np.dot(ps, xs)


dean_mean = pmf_mean(pmf_class_sizes)
print("Avg class size according to the Dean: %.3f" % (dean_mean))

[0.12307692 0.         0.         0.         0.         0.12307692
 0.         0.         0.         0.         0.21538462 0.
 0.         0.         0.         0.06153846 0.         0.
 0.         0.         0.09230769 0.         0.         0.
 0.         0.18461538 0.         0.         0.         0.
 0.12307692 0.         0.         0.         0.         0.04615385
 0.         0.         0.         0.03076923]
[ 7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19. 20. 21. 22. 23. 24.
 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36. 37. 38. 39. 40. 41. 42.
 43. 44. 45. 46. 47.]
Avg class size according to the Dean: 23.662


/home/venkat/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  """Entry point for launching an IPython kernel.


In [53]:
# extract and shuffle population
actual_pop = []
for key in class_size_dist.keys():
    for i in range(class_size_dist[key]):
        actual_pop.append(key)
pop = np.array(actual_pop)
np.random.shuffle(pop)

# sample from this population for 1/10-th the size
sum1 =0
polled_sample = []
polled_mean = 0
pop_size = pop.shape[0]
sample_size =  int(pop_size * 0.1)
for i in range(sample_size):
    dice = np.random.randint(pop_size)
    polled_sample.append(pop[dice])
    print(pop[dice])


for i in range(len(polled_sample)):
    sum1 = sum1 + polled_sample[i]



polled_mean = sum1 / len(polled_sample)

print("Avg class size according to polling: %d" % polled_mean)

27
27
32
32
17
37
Avg class size according to polling: 28


The mean calculated from pmf and mean calculated by choosing the random student classes is different because larger classes get oversampled

## Birth Weights

PMFs work well if the number of values is small, but as the number increases, the probability associated with each value gets smaller and smaller and the effect of random noise increases.

In [ ]:
pregnancies = pd.read_fwf("../data/2002FemPreg.dat", 
                         names=["caseid", "nbrnaliv", "babysex", "birthwgt_lb",
                               "birthwgt_oz", "prglength", "outcome", "birthord",
                               "agepreg", "finalwgt"],
                         colspecs=[(0, 12), (21, 22), (55, 56), (57, 58), (58, 60),
                                (274, 276), (276, 277), (278, 279), (283, 285), (422, 439)])
pregnancies.head()

In [ ]:
live_births = pregnancies[pregnancies["outcome"] == 1]

first_babies_df = live_births[live_births["birthord"] == 1]
first_babies_df["totwgt"] = first_babies_df["birthwgt_lb"] * 16 + \
    first_babies_df["birthwgt_oz"]
other_babies_df = live_births[live_births["birthord"] != 1]
other_babies_df["totwgt"] = other_babies_df["birthwgt_lb"] * 16 + \
    other_babies_df["birthwgt_oz"]

first_babies = np.array(first_babies_df["totwgt"].dropna())
other_babies = np.array(other_babies_df["totwgt"].dropna())

range_lb = int(min([np.min(first_babies), np.min(other_babies)]))
range_ub = int(max([np.max(first_babies), np.max(other_babies)]))
nbr_bins = range_ub - range_lb

pmf_first_babies = np.histogram(np.array(first_babies), 
                                bins=nbr_bins, range=(range_lb, range_ub), normed=True)
pmf_other_babies = np.histogram(np.array(other_babies), 
                                bins=nbr_bins, range=(range_lb, range_ub), normed=True)


## Percentiles

If you are given a value, it is easy to find its percentile rank; going the other
way is slightly harder. If you are given a percentile rank and you want to
find the corresponding value, one option is to sort the values and search for
the one you want:

In [71]:
def percentile_rank(xs, score):
    count = 0
    for x in xs:
        if x < score:
            count += 1
    return 100.0 * count / len(xs)

def percentile(xs, rank):
    sxs = sorted(xs)
    for x in sxs:
        if percentile_rank(sxs, x) >= rank:
            return x

scores = np.array([55, 66, 77, 88, 99])
print("Percentile rank for 88: %d" % (percentile_rank(scores, 88)))
print("Percentile for rank 60: %d" % (percentile(scores, 60)))



def Percentile2(scores, percentile_rank):
 
    scores.sort()
    index = percentile_rank * (len(scores)-1) / 100
    return scores[floor(index)]

def floor(val):
    if type(val)== float:
        n = int(val)
        if n > val-1 and n < val + 1:
            return n+1
    else:
        return val
    
    
print(Percentile2(scores,60))


4
88


# CDF
Commulative Distributive Function

The CDF is the function that maps values to their percentile rank in a distribution.

The CDF is a function of x, where x is any value that might appear in the distribution. To evaluate CDF(x) for a particular value of x, we compute the
fraction of the values in the sample less than (or equal to) x.

The result is in a probability in the range 0–1 rather than a percentile
rank in the range 0–100




In [75]:
count = 0
def Cdf(t, x):
    count = 0.0
    for value in t:
        if value <= x:
            count += 1.0
    prob = count / len(t)
    return prob

t = [55,66,77,88,99]
Cdf(t,88)

0.8

### CDF for the survey data

According to the suvery data when cdf of  birthweight is calculated there is no much difference between first borns and others before mean.
And after mean there is larger discrepancy above the mean.

In [ ]:
def cdf(pmf):
    xs = pmf[1][:-1]
    ps = np.cumsum(pmf[0])
    return np.vstack((xs, ps)).T

cdf_first_babies = cdf(pmf_first_babies)
cdf_other_babies = cdf(pmf_other_babies)

In [ ]:
xs = cdf_first_babies[:, 0]
ps = cdf_first_babies[:, 1]

# given the weight find the percentile
birth_weight = (7 * 16) + 9
for i in range(xs.shape[0]):
    if xs[i] >= birth_weight:
        break
pc_rank = ps[i] * 100
print("Percentile rank of first baby 7 lb 9 oz: %.3f" % (pc_rank))

# given the percentile find the weight
percentile = 75
for i in range(ps.shape[0]):
    if ps[i] > 0.01 * percentile:
        break
weight = xs[i]
print("Weight of first born in the 75th percentile: %.3f oz" % (weight))

## Conditional Distributions

Distribution of a subset of data which is selected according to a condition.

For example, if you are above average in weight, but way above average in
height, then you might be relatively light for your height. Here’s how you could make that claim more precise.
1. Select a cohort of people who are the same height as you (within some range).
2. Find the CDF of weight for those people.
3. Find the percentile rank of your weight in that distribution.



In [80]:
overall_percentile_rank = 97.0 * 100 / 1633
division_percentile_rank = 26.0 * 100 / 256
print("overall percentile rank: %.3f, percentile rank in division: %.3f" % 
      (overall_percentile_rank, division_percentile_rank))

overall percentile rank: 5.940, percentile rank in division: 10.156


## Random Numbers

CDFs are useful for generating random numbers with a given distribution.
Here’s how:
 1: Choose a random probability in the range 0–1.
 2: Use Cdf.Value to find the value in the distribution that corresponds to the probability you chose


The process of generating a sample based on another sample is called resampling. Resampling can be with replacement  and without replacement.

In [ ]:
sample_size = 100
samples = []
for i in range(sample_size):
    prob = np.random.random()
    for j in range(ps.shape[0]):
        if ps[j] > prob:
            break
    samples.append(xs[j])

# create a CDF for the new sample
pmf_sample = np.histogram(np.array(samples), 
                          bins=nbr_bins, range=(range_lb, range_ub), normed=True)
cdf_sample = cdf(pmf_sample)


## Summary Statistics Revisited

The median is just the 50th percentile. The 25th and 75th percentiles are often used to check whether a distribution is symmetric, and their difference, which is called the interquartile range, measures the spread.
it is also called the midspread or middle 50%.

We find the median and IQR using the CDF for the firstborn birth weights.

In [ ]:
xs = cdf_first_babies[:, 0]
ps = cdf_first_babies[:, 1]

# median
for i in range(ps.shape[0]):
    if ps[i] >= 0.5:
        break
print("median = %.3f" % (xs[i]))

# IQR
for i in range(ps.shape[0]):
    if ps[i] >= 0.25:
        break
q1 = xs[i]
for i in range(ps.shape[0]):
    if ps[i] >= 0.75:
        break
q3 = xs[i]
iqr = q3 - q1
print("Q1 = %.3f, Q3 = %.3f, IQR = %.3f" % (q1, q3, iqr))